In [2]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import time, os, shutil, re
import requests

In [4]:
# investing.com 이게 크롤링 돌아가는 코드
main_path = 'investing_crawl'


main = 'https://www.investing.com'
topics = ['stock-market-news', 'commodities-news', 'forex-news', 'economy', 'economic-indicators','politics','world-news']

for topic in topics[6:]:
    title_lst = []
    content_lst = []
    date_lst = []
    company_lst = []
    save_path = '/'.join([main_path, topic])
    if not os.path.exists(save_path):
        os.mkdir(save_path)
    url = '/'.join([main,'news', topic])
    headers = {
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36',
            'Referer': '/'.join([main, 'news', topic])}
    res = requests.get(url,headers=headers)
    time.sleep(0.2)
    print(topic, int(BeautifulSoup(res.text, 'html.parser').select('#paginationWrap > div.midDiv.inlineblock > a:nth-child(5)')[0]['title'].split()[-1])//17)
    for page in range(2701, int(BeautifulSoup(res.text, 'html.parser').select('#paginationWrap > div.midDiv.inlineblock > a:nth-child(5)')[0]['title'].split()[-1])//17):
        print(page)
        url = '/'.join([main,'news', topic, str(page)])
        headers = {
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36',
            'Referer': '/'.join([main, 'news', topic, str(page)])}
        res = requests.get(url,headers=headers)
        time.sleep(0.2)
        soup = BeautifulSoup(res.text, 'html.parser')
        article_path = '#leftColumn > div.largeTitle > article'
        nlstdir = soup.select(article_path)
        if not nlstdir:
            print('done')
            break
        for newslst in nlstdir:
            title = np.nan
            contents = np.nan
            date = np.nan
            try:
                if 'dfp-native' in newslst['class']:
                    continue
                elif 'https://' in newslst.select('a')[0]['href']:
                    newsurl = newslst.select('a')[0]['href']
                else : 
                    newsurl = main+newslst.select('a')[0]['href']
                newsres = requests.get(newsurl, headers = headers)
                time.sleep(0.2)
                news = BeautifulSoup(newsres.text,'html.parser')
                if newslst.select('div.textDiv > span > span:nth-child(1)'):
                    company = newslst.select('div.textDiv > span > span:nth-child(1)')[0].text
                    if company == 'By Investing.com Studios':
                        continue
                    title = news.select('#leftColumn > h1')[0].text
                    date = news.select('#leftColumn > div.contentSectionDetails > span')[0]
                    contents = []
                    for con in news.select('#leftColumn > div.articlePage > p'):
                        contents.append(con.text)
                    contents = '\n'.join(contents)

                elif newslst.select('div.textDiv > div > span:nth-child(1)'):
                    company = newslst.select('div.textDiv > div > span:nth-child(1)')[0].text
                    # 각 회사별 크롤링 돌아가게하는 코드 작성해야함
                    if company == 'By CNBC': # Sample#1 : CNBC
                        title = news.select('#main-article-header > div:nth-child(1)>div:nth-child(1)>div:nth-child(2)')[0].text
                        date = news.select('#main-article-header > div:nth-child(1)>div:nth-child(1)>div:nth-child(3)>time')[0]['datetime']
                        contents = []
                        for con in news.select('#RegularArticle-ArticleBody-5 > div.group'):
                            contents.append(con.text)
                        contents = '\n'.join(contents)
                    # CNBC 랑 비슷하게 contents title date 받을수 있게 코드좀 만들어주세요
                    # 각 회사별 링크는 직접 찾아서 들어가봐야합니다.. 아래가 종류
                    elif company == 'By CryptoDaily': # disclaimer 후처리해야함
                        title = news.select('#PostsPage > div.bg-body.box-layout > h2')[0].text
                        date = news.select('#PostsPage > div.bg-body.box-layout > div.row.article-meta > div > div > div > div.date-count')[0].text
                        contents = []
                        for lst in news.select('div.news-content'):
                            contents.append(lst.text)
                        contents = '\n'.join(contents)
                    elif company == 'By CryptoGlobe':
                        title = news.select('#content_start > div:nth-child(1) > article > header > div:nth-child(1) > h1')[0].text
                        date = news.select('#content_start > div:nth-child(1) > article > header > div.row > div:nth-child(1) > div > div:nth-child(2) > ul > li:nth-child(1)')[0].text
                        contents = news.select('div.article-body')[0].text
                    elif company == 'By Huffington Post':
                        title = news.select('#entry-header > header > div.top-header > h1')[0].text
                        date = news.select('#entry-header > header > div.bottom-header > div.bottom-header__left > div.timestamp > time:nth-child(1) > span')[0].text
                        divs = news.select('#entry-body > section')[0].text
                        contents = []
                        for i in divs.split('\n'):
                            contents.append(i.strip('Advertisement'))
                        contents = '\n'.join(contents)
                    else:
                        pass
                else:
                    print(f'Error Occured in {topic} page:{page}')
                    pass
            except:
                print(f'Error Occured in {topic} page:{page}')
                continue
            company_lst.append(company)
            title_lst.append(title)
            date_lst.append(date)
            content_lst.append(contents)
        if page%100 == 0:
            pd.DataFrame({'company':company_lst, 'date':date_lst, 'title':title_lst, 'content':content_lst}).to_csv(save_path+f'/news_{topic}_{page//100:05}.csv')
            title_lst = []
            content_lst = []
            date_lst = []
            company_lst = []
    pd.DataFrame({'company':company_lst, 'date':date_lst, 'title':title_lst, 'content':content_lst}).to_csv(save_path+f'/news_{topic}_{page//100+1:05}.csv')

world-news 7244
2701
2702
2703
2704
2705
2706
2707
2708
2709
2710
2711
2712
2713
2714
2715
2716
2717
2718
2719
2720
2721
2722
2723
2724
2725
2726
2727
2728
2729
2730
2731
2732
2733
2734
2735
2736
2737
2738
2739
2740
2741
2742
2743
2744
2745
2746
2747
2748
2749
2750
2751
2752
2753
2754
2755
2756
2757
2758
2759
2760
2761
2762
2763
2764
2765
2766
2767
2768
2769
2770
2771
2772
2773
2774
2775
2776
2777
2778
2779
2780
2781
2782
2783
2784
2785
2786
2787
2788
2789
2790
2791
2792
2793
2794
2795
2796
2797
2798
2799
2800
2801
2802
2803
2804
2805
2806
2807
2808
2809
2810
2811
2812
2813
2814
2815
2816
2817
2818
2819
2820
2821
2822
2823
2824
2825
2826
2827
2828
2829
2830
2831
2832
2833
2834
2835
2836
2837
2838
2839
2840
2841
2842
2843
2844
2845
2846
2847
2848
2849
2850
2851
2852
2853
2854
2855
2856
2857
2858
2859
2860
2861
2862
2863
2864
2865
2866
2867
2868
2869
2870
2871
2872
2873
2874
2875
2876
2877
2878
2879
2880
2881
2882
2883
2884
2885
2886
2887
2888
2889
2890
2891
2892
2893
2894
2895
2896
2897

ConnectionError: ('Connection aborted.', ConnectionAbortedError(10053, '현재 연결은 사용자의 호스트 시스템의 소프트웨어의 의해 중단되었습니다', None, 10053, None))

In [111]:
newslst['class']

['articleItem', 'dfp-native']

In [116]:
pd.read_csv('D:/Project4/data/investing_crawl/stock-market-news/news_stock-market-news_00001.csv',index_col=0).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   company  103 non-null    object
 1   date     99 non-null     object
 2   title    99 non-null     object
 3   content  98 non-null     object
dtypes: object(4)
memory usage: 4.0+ KB


In [88]:
news = BeautifulSoup(requests.get("https://invst.ly/xxtbk").text, 'html.parser')
news.select('#main-article-header > div > div.ArticleHeader-wrapperHeroNoImage.ArticleHeader-wrapperHero.ArticleHeader-wrapper.ArticleHeader-wrapperNoImage > div.ArticleHeader-time > time')

[]

In [96]:
print(news.select('#content_start > div:nth-child(1) > article > header > div:nth-child(1) > h1')[0].text)
print(news.select('#content_start > div:nth-child(1) > article > header > div.row > div:nth-child(1) > div > div:nth-child(2) > ul > li:nth-child(1)')[0].text)
for lst in news.select('div.article-body'):
    continue
print(lst.text)

$USDC: World’s Largest Asset Manager With $9.5 Trillion AUM Interested in USD Coin
16 Apr 2022

NYC-based BlackRock Inc., which is the world’s top asset management firm with around $9.5 trillion in total assets under management (AUM) as of 30 September 2021 according to ADV Ratings, has invested in stablecoin issuer Circle Internet Financial Ltd.
On 26 September 2018, Circle announced that 26 September 2018 announced that it was launching its “USD Coin” ($USDC for short), which is a regulated fully collateralized dollar-backed cryptocurrency that was originally announced on 16 May 2018. This stablecoin is based on an open source fiat stablecoin framework developed and governed by the CENTRE project. Circle said back then that “individuals and institutions” could enroll its dollar tokenization service to “deposit US dollars from bank accounts, convert those dollars into tokens usable everywhere the internet reaches (subject to the token’s compliance controls), and redeem USDC tokens and

In [97]:
t = 100
f'{t:05}'

'00100'